In [1]:
import pandas as pd
from math import exp
import numpy as np
import math
from itertools import permutations, combinations, product
from functools import reduce


Parameters for Group 29

$u = 1.1 + \frac{29}{100} = 1.39$

$d = \frac{1}{u} = 0.7194$

$p^* = \frac{1 - d}{u - d} = 0.4184$


In [3]:
S = 95  # stock underlying value
K = 105 # strike
t = 1.0 # time
r = 0.0 # risk-free interest rate
n = 3 # number of steps
v = 0.0 # volatility
T = t * n
u = 1.39
d = 1 / u
At = t / n 
p = (1 - d) / (u - d) 

# Part 1 and 2

In [4]:
def buildtree_Call(u, d, T, S, K):
    
    # Generate the paths
    a=product('ud', repeat=T) 
    paths = []
    for i in a:
        paths.append(''.join(i))
    
    # Calculate the paths
    b=product([u,d], repeat=T) 
    vals= []
    for i in b:
        vals.append(list(i))
    vals1 = np.matrix(vals)
    
    # Create table for the paths
    # generate first column
    startingvals = np.repeat(S, 2**T)
    rv = startingvals.reshape(2**T,1)
    # generate subsequent columns by multiplying the values of previous columns
    stockvalues = []
    stockvalues.append(rv.T.tolist()[0])
    for i in range(T):
        rv = np.multiply(rv, vals1[:,i])
        stockvalues.append(rv.T.tolist()[0])   
    
    # Crate a dataframe from the matrix of path values
    stockvalues = np.array(stockvalues)
    dfstock = pd.DataFrame(stockvalues.T)
    
    # Calculate the last column which is the payoff
    dfstock['H'] = dfstock[T].apply(lambda x: x-K if x-K > 0 else 0)
    dfstock.index = paths
    return dfstock


In [5]:
def buildtree_Put(u, d, T, S, K):
    
    # Generate the paths
    a=product('ud', repeat=T) 
    paths = []
    for i in a:
        paths.append(''.join(i))
    
    # Calculate the paths
    b=product([u,d], repeat=T) 
    vals= []
    for i in b:
        vals.append(list(i))
    vals1 = np.matrix(vals)
    
    # Create table for the paths
    # generate first column
    startingvals = np.repeat(S, 2**T)
    rv = startingvals.reshape(2**T,1)
    # generate subsequent columns by multiplying the values of previous columns
    stockvalues = []
    stockvalues.append(rv.T.tolist()[0])
    for i in range(T):
        rv = np.multiply(rv, vals1[:,i])
        stockvalues.append(rv.T.tolist()[0])   
    
    # Crate a dataframe from the matrix of path values
    stockvalues = np.array(stockvalues)
    dfstock = pd.DataFrame(stockvalues.T)
    
    # Calculate the last column which is the payoff
    dfstock['H'] = dfstock[T].apply(lambda x: x-K if x-K < 0 else 0)
    dfstock.index = paths
    return dfstock


In [6]:
def pieh(u, d, df, col=None, K=None):

    if col is None or col == 'H':
        hvals = df['H'].to_list()
    else:
        if col not in df.columns:
            return np.nan
        else:
            hvals = df[col].apply(lambda x: x-K if x-K < 0 else 0).to_list() 
                
    # Calculate the p*
    p = (1 - d) / (u - d) 
    q = 1 - p
    pieH = 0
    T = int(np.log2(len(df)))
    
    # get the product structure p^m q^n
    t1 = perm(p, q, T)
    # reshape the H values to perform matrix multiplication
    h = np.round(hvals, 2)
    h2 = h.reshape(len(h), 1)
    prod1 = np.multiply(t1, h2)
    # unique no-arbitrage price of H is
    pieH = np.sum(prod1)
    return pieH

def perm(p, q, T):
    b=product((p,q), repeat=T) 
    vals= []
    for i in b:
        vals.append(list(i))
    vals1 = np.matrix(vals)
    t1 = reduce(lambda a,b: np.multiply(a,b), [m for m in vals1.T])
    return t1.T
    

#### Call option S = 95, K = 105, T = 3

In [7]:
df3 = buildtree_Call(u=1.39, d=1/1.39, T=3, S=95, K=105)
df3

,0,1,2,3,H
uuu,95.0,132.050000,183.549500,255.133805,150.133805
uud,95.0,132.050000,183.549500,132.050000,27.050000
udu,95.0,132.050000,95.000000,132.050000,27.050000
udd,95.0,132.050000,95.000000,68.345324,0.000000
duu,95.0,68.345324,95.000000,132.050000,27.050000
dud,95.0,68.345324,95.000000,68.345324,0.000000
ddu,95.0,68.345324,49.169298,68.345324,0.000000
ddd,95.0,68.345324,49.169298,35.373595,0.000000


#### Put option S = 95, K = 105, T = 2

In [8]:
u=1.39
d=1/u
T=2
S=95
K=105
p = (1 - d) / (u - d) 
q = 1 - p

In [9]:
df_put = buildtree_Put(u=9, d=1/1.39, T=2, S=95, K=105)
df_put

,0,1,2,H
uu,95.0,855.000000,7695.000000,0.000000
ud,95.0,855.000000,615.107914,0.000000
du,95.0,68.345324,615.107914,0.000000
dd,95.0,68.345324,49.169298,-55.830702


The price of the option - unique no-arbitrage price of H

In [10]:
pieh(u=1.39, d=1/1.39, df=df_put, col='H', K=105)

-18.884323278654087

##### Check put-call parity

In [12]:
df_call = buildtree_Call(u=1.39, d=1/1.39, T=2, S=95, K=105)
df_call

,0,1,2,H
uu,95.0,132.050000,183.549500,78.5495
ud,95.0,132.050000,95.000000,0.0000
du,95.0,68.345324,95.000000,0.0000
dd,95.0,68.345324,49.169298,0.0000


In [13]:
np.round(pieh(u=1.39, d=1/1.39, df=df_call, col='H', K=105)+pieh(u=1.39, d=1/1.39, df=df_put, col=2, K=105), 2)

-5.13

This is equal to the difference S - K = 95 - 105 = -10 

## Value of the replicating strategy

In [14]:
def repli_val(p, q, df, n):
    T = int(np.log2(len(df)))
    if n >= T:
        return -1
    
    p1 = perm(p, q, T-n)
    
    l = len(p1)
    j = 0
    sum = 0 
    v = []
    for i, val in enumerate(df['H']):
        sum += p1[j]*val
        j += 1
        if j >= l:
            j = 0
            v.append([sum]*l)
            sum = 0
    return np.array(v ).reshape(2**T,1)

def replicating_strategy(p, q, df_option):
    df = pd.DataFrame()

    for i in range(T):
        v = repli_val(p, q, df_option, i)
        df[i] = v.T.tolist()[0]
    df[i+1] = df_option.reset_index()['H']
    df.index = df_option.index
    return df
        

In [15]:
u=1.39
d=1/u
T=2
S=95
K=105

df_v = replicating_strategy(p, q, df_put)
df_v

,0,1,2
uu,-18.884561,0.000000,0.000000
ud,-18.884561,0.000000,0.000000
du,-18.884561,-32.470576,0.000000
dd,-18.884561,-32.470576,-55.830702


In [16]:
df_put

,0,1,2,H
uu,95.0,855.000000,7695.000000,0.000000
ud,95.0,855.000000,615.107914,0.000000
du,95.0,68.345324,615.107914,0.000000
dd,95.0,68.345324,49.169298,-55.830702


### Hedging strategy

In [17]:
def hedging_strategy(df_v, df_x):
    if len(df_v.columns) != len(df_x.columns):
        return -1
    phi = []
    for i in range(1, len(df_v.columns)):
        ph1 = np.round(((df_v[i] - df_v[i-1])/(df_x[i] - df_x[i-1])).iloc[0], 2)
        phi.append(ph1)
    return phi

def xvalues(df):
    return df.drop(columns='H')

In [18]:
df_x = xvalues(df_put)

In [19]:
hedging_strategy(df_v, df_x)

[0.02, 0.0]

# Part 3

The no-arbitrage condition requires  
$$V \cdot \phi \geq 0 $$ isa null set, where V is the value of he portfolio and $\phi$ is the trading strategy (portfolio allocation).  
Let's set $V \cdot \phi = H$ where $H$ is the value of the option with acts a replicating strategy and is a constant. 
If there is a solution to this equation then there is no arbitrage opportuniies. We can rewite this equation as Ax = b and try to solve for x.

In [25]:
A = np.array([[1,15], [1,5]])
A

array([[ 1, 15],
       [ 1,  5]])

In [21]:
b = np.array([[3,1]]).T
b

array([[3],
       [1]])

In [22]:
np.linalg.inv(A)

array([[-0.5,  1.5],
       [ 0.1, -0.1]])

The solution for x

In [29]:
np.dot(np.linalg.inv(A), b)

array([[0. ],
       [0.2]])